# Exploring Early SARS-CoV2 Mutations
We are working in the directory *SARS-CoV-2*.

## Install Programs

We need two AWK scripts

In [10]:
wget 'https://github.com/awkologist/CompBiol3/raw/main/SARS-CoV-2/fasta2tbl'

--2024-02-10 10:58:47--  https://github.com/awkologist/CompBiol3/raw/main/SARS-CoV-2/fasta2tbl
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/awkologist/CompBiol3/main/SARS-CoV-2/fasta2tbl [following]
--2024-02-10 10:58:47--  https://raw.githubusercontent.com/awkologist/CompBiol3/main/SARS-CoV-2/fasta2tbl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 299 [text/plain]
Saving to: ‘fasta2tbl.1’

fasta2tbl.1         100%[===================>]     299  --.-KB/s    in 0s      

2024-02-10 10:58:47 (586 KB/s) - ‘fasta2tbl.1’ saved [299/299]



In [9]:
wget 'https://github.com/awkologist/CompBiol3/raw/main/SARS-CoV-2/compare-cov2.awk'

--2024-02-10 10:44:06--  https://github.com/awkologist/CompBiol3/raw/main/SARS-CoV-2/compare-cov2.awk
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/awkologist/CompBiol3/main/SARS-CoV-2/compare-cov2.awk [following]
--2024-02-10 10:44:06--  https://raw.githubusercontent.com/awkologist/CompBiol3/main/SARS-CoV-2/compare-cov2.awk
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2904 (2.8K) [text/plain]
Saving to: ‘compare-cov2.awk.1’

compare-cov2.awk.1  100%[===================>]   2.84K  --.-KB/s    in 0.003s  

2024-02-10 10:44:07 (855 KB/s) - ‘compare-cov2.awk.1’ saved [2904/2904]



We also need the molecular structure viewer **Jmol**

In [ ]:
sudo apt install -y jmol

## Download Virus Sequences

Download reference genome:

In [1]:
efetch -db nuccore -id NC_045512 -format fasta > wuhan-1.fasta

We create a copy in tab-delimited format:

In [13]:
./fasta2tbl wuhan-1.fasta > wuhan-1.tab

Download from [NCBI](https://www.ncbi.nlm.nih.gov/sars-cov-2/) viruses from Europe, from human hosts, without ambigious characters, complete nucleotide sequences, and a sequence length of exactly 29,903 nt. They are downloaded via the web browser as *sequences.fasta*. Move them into your current working directory.

In [1]:
cut -f 1 wuhan-1.tab

NC_045512.2 Severe acute respiratory syndrome coronavirus 2 isolate Wuhan-Hu-1, complete genome


Rename the file to *cov2-len-29903.fasta*

In [2]:
mv sequences.fasta cov2-len-29903.fasta

In [3]:
grep -c ">" cov2-len-29903.fasta

3371


## Convert FASTA to TAB and Edit Header

In [9]:
head -1 sequences.fasta

>OY902096.1 |United Kingdom:England|2020-10-12


In [2]:
./fasta2tbl sequences.fasta | sed 's/|/\t/g' > cov2-len-29903.tab

In [4]:
wc -l cov2-len-29903.tab

3371 cov2-len-29903.tab


In [5]:
cut -f 1 cov2-len-29903.tab | head -2

OY902096.1 
OX648031.1 


In [6]:
cut -f 1,3 cov2-len-29903.tab | head -2

OY902096.1 	2020-10-12
OX648031.1 	2020-11-01


In [7]:
cut -f 1-3 cov2-len-29903.tab | head -2

OY902096.1 	United Kingdom:England	2020-10-12
OX648031.1 	United Kingdom:England	2020-11-01


## Analyze Data

In [10]:
cut -f 2 cov2-len-29903.tab | sed 's/:.*//' | sort | uniq -c 

     82 France
      7 Germany
      2 Greece
      1 Italy
     14 Poland
      7 Russia
      1 Serbia
      8 Spain
      1 Sweden
   3248 United Kingdom


In [11]:
egrep "Germany" cov2-len-29903.tab | cut -f 1-3

OK075090.1 	Germany: Langen	2021-01-08
MT358638.1 	Germany	2020-02
MT358639.1 	Germany	2020-02
MT358640.1 	Germany	2020-02
MT358641.1 	Germany	2020-02
MT358642.1 	Germany	2020-02
MT358643.1 	Germany	2020-02


In [18]:
awk -f compare-cov2.awk -v ref=wuhan-1.tab -v seq=cov2-len-29903.tab -v id2=MT358638

# Compared: NC_045512.2 vs MT358638.1  | 12/2019 vs 2020-02 | Wuhan vs Germany
MT358638.1  3518: G>T 
MT358638.1  12704: G>T 
MT358638.1  12797: G>A 
MT358638.1  17423: A>G 
MT358638.1  27272: T>C 
MT358638.1  28854: C>T 
# of N's: 0 of 29903
# of exchanges: 6 of 29903


In [20]:
for i in MT358638 MT358639 MT358640 MT358641 MT358642 MT358643; do awk -f compare-cov2.awk -v ref=wuhan-1.tab -v seq=cov2-len-29903.tab -v id2=$i; done

# Compared: NC_045512.2 vs MT358638.1  | 12/2019 vs 2020-02 | Wuhan vs Germany
MT358638.1  3518: G>T 
MT358638.1  12704: G>T 
MT358638.1  12797: G>A 
MT358638.1  17423: A>G 
MT358638.1  27272: T>C 
MT358638.1  28854: C>T 
# of N's: 0 of 29903
# of exchanges: 6 of 29903
# Compared: NC_045512.2 vs MT358639.1  | 12/2019 vs 2020-02 | Wuhan vs Germany
MT358639.1  241: C>T 
MT358639.1  3037: C>T 
MT358639.1  14408: C>T 
MT358639.1  23403: A>G SPIKE NT:1841 AA:614 -> D
MT358639.1  28881: G>A 
MT358639.1  28882: G>A 
MT358639.1  28883: G>C 
# of N's: 0 of 29903
# of exchanges: 7 of 29903
# Compared: NC_045512.2 vs MT358640.1  | 12/2019 vs 2020-02 | Wuhan vs Germany
MT358640.1  241: C>T 
MT358640.1  3037: C>T 
MT358640.1  6228: A>G 
MT358640.1  14408: C>T 
MT358640.1  16289: C>Y 
MT358640.1  23403: A>G SPIKE NT:1841 AA:614 -> D
MT358640.1  28881: G>A 
MT358640.1  28882: G>A 
MT358640.1  28883: G>C 
MT358640.1  28961: C>T 
MT358640.1  29870: C>M 
# of N's: 0 of 29903
# of exchanges: 11 of 29903


In [21]:
awk -f compare-cov2.awk -v ref=wuhan-1.tab -v seq=cov2-len-29903.tab -v id2=OK075090

# Compared: NC_045512.2 vs OK075090.1  | 12/2019 vs 2021-01-08 | Wuhan vs Germany: Langen
OK075090.1  241: C>T 
OK075090.1  1059: C>T 
OK075090.1  3037: C>T 
OK075090.1  5143: T>G 
OK075090.1  7843: C>T 
OK075090.1  14408: C>T 
OK075090.1  15920: T>C 
OK075090.1  21846: C>T SPIKE NT:284 AA:95 -> T
OK075090.1  22296: A>G SPIKE NT:734 AA:245 -> H
OK075090.1  23403: A>G SPIKE NT:1841 AA:614 -> D
OK075090.1  23618: A>G SPIKE NT:2056 AA:686 -> S
OK075090.1  26895: C>T 
OK075090.1  27996: G>T 
OK075090.1  28044: G>T 
OK075090.1  29769: C>T 
# of N's: 0 of 29903
# of exchanges: 15 of 29903
